# 7b – Road‑network travel time

Use **road‑network travel time** (via *OSMnx* + *NetworkX*) rather than straight‑line distance.  
Each CDC/CH is linked to the acute hospital reachable in the shortest number of minutes by car (default 50 km/h if speed data absent).  

In [ ]:

import pandas as pd
import osmnx as ox
import networkx as nx
import math
from pathlib import Path

DATA_DIR = Path('.')
ACUTE_CSV = DATA_DIR / 'NHS_SW_Acute_Hospitals_enriched.csv'
CDC_CSV   = DATA_DIR / 'NHS_SW_Community_Diagnostic_Centres_enriched.csv'
CH_CSV    = DATA_DIR / 'NHS_SW_Community_Hospitals_enriched.csv'

acute = pd.read_csv(ACUTE_CSV)
cdc   = pd.read_csv(CDC_CSV)
ch    = pd.read_csv(CH_CSV)

# -------------------------------------------------------------------------
# Build or load a drivable road network for the region.
# **Tip:** the first call to `graph_from_place` may take minutes and require
# internet; cache with `ox.save_graphml()` for reuse.
# -------------------------------------------------------------------------
G = ox.graph_from_place('South West England, United Kingdom',
                        network_type='drive', simplify=True)

def travel_time_min(graph, lat1, lon1, lat2, lon2, default_speed_kmh=50):
    """Return shortest travel time (minutes) using edge lengths & speeds."""
    orig_node = ox.nearest_nodes(graph, lon1, lat1)
    dest_node = ox.nearest_nodes(graph, lon2, lat2)
    try:
        length_m = nx.shortest_path_length(graph, orig_node, dest_node,
                                           weight='length')
    except nx.NetworkXNoPath:
        return math.inf
    return (length_m / 1000) / default_speed_kmh * 60  # minutes

def label_nearest_time(df, hubs):
    nearest, minutes = [], []
    for _, spoke in df.iterrows():
        times = hubs.apply(
            lambda hub: travel_time_min(G, spoke.latitude, spoke.longitude,
                                        hub.latitude, hub.longitude), axis=1)
        idx = times.idxmin()
        nearest.append(hubs.loc[idx, 'Name'])
        minutes.append(times.iloc[idx])
    out = df.copy()
    out['nearest_acute'] = nearest
    out['travel_min'] = minutes
    return out

cdc = label_nearest_time(cdc, acute)
ch  = label_nearest_time(ch,  acute)

cdc[['Name', 'nearest_acute', 'travel_min']].head()
